In [7]:
%pip install -q sympy gmpy2 pycryptodome pyprimes primefac pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sympy
import gmpy2
from Crypto.Util.number import isPrime as _crypto_isPrime
import pyprimes
import primefac


def is_prime_sympy(n: int) -> bool:
    try:
        return bool(sympy.isprime(n))
    except Exception:
        return False


def is_prime_gmpy2(n: int) -> bool:
    try:
        return bool(gmpy2.is_prime(gmpy2.mpz(n)))
    except Exception:
        return False


def is_prime_pycryptodome(n: int) -> bool:
    try:
        return bool(_crypto_isPrime(n))
    except Exception:
        return False


def is_prime_pyprimes(n: int) -> bool:
    try:
        return bool(pyprimes.isprime(n))
    except Exception:
        return False


def is_prime_primefac(n: int) -> bool:
    try:
        return bool(primefac.isprime(n))
    except Exception:
        return False

In [ ]:
import sys

_PRIME_NUMBERS_LIST = []
_NON_PRIME_NUMBERS_LIST = []

i = sys.maxsize
while i >= 0 and (len(_PRIME_NUMBERS_LIST) < 10 or len(_NON_PRIME_NUMBERS_LIST) < 10):
    is_prime = sympy.isprime(i)
    if is_prime and len(_PRIME_NUMBERS_LIST) < 10:
        _PRIME_NUMBERS_LIST.append(i)
    elif not is_prime and len(_NON_PRIME_NUMBERS_LIST) < 10:
            _NON_PRIME_NUMBERS_LIST.append(i)

    i -= 1

_PRIME_NUMBERS_LIST.reverse()
_NON_PRIME_NUMBERS_LIST.reverse()

print("Prime Numbers for Testing:", _PRIME_NUMBERS_LIST)
print("Non-Prime Numbers for Testing:", _NON_PRIME_NUMBERS_LIST)

Prime Numbers for Testing: [9223372036854775337, 9223372036854775351, 9223372036854775399, 9223372036854775417, 9223372036854775421, 9223372036854775433, 9223372036854775507, 9223372036854775549, 9223372036854775643, 9223372036854775783]
Non-Prime Numbers for Testing: [9223372036854775798, 9223372036854775799, 9223372036854775800, 9223372036854775801, 9223372036854775802, 9223372036854775803, 9223372036854775804, 9223372036854775805, 9223372036854775806, 9223372036854775807]


In [9]:
import time
from statistics import mean
import pandas as pd

funcs = [
    is_prime_sympy,
    is_prime_gmpy2,
    is_prime_pycryptodome,
    is_prime_pyprimes,
    is_prime_primefac,
]

# Adjust if you want more stable timings (will run slower)
REPEATS_PER_INPUT = 1

def _bench_one(func, n):
    t0 = time.perf_counter_ns()
    out = None
    for _ in range(REPEATS_PER_INPUT):
        out = func(n)
    t1 = time.perf_counter_ns()
    # average per call
    return out, (t1 - t0) / REPEATS_PER_INPUT

def _bench_list(func, nums, expected):
    correct = 0
    times_ns = []
    for n in nums:
        out, dt_ns = _bench_one(func, n)
        times_ns.append(dt_ns)
        correct += int(out is expected)
    acc_pct = 100.0 * correct / len(nums) if nums else float("nan")
    avg_ns = mean(times_ns) if times_ns else float("nan")
    return acc_pct, avg_ns

rows = []
for f in funcs:
    prime_acc, prime_avg_ns = _bench_list(f, _PRIME_NUMBERS_LIST, True)
    nonprime_acc, nonprime_avg_ns = _bench_list(f, _NON_PRIME_NUMBERS_LIST, False)
    both_avg_ns = mean([prime_avg_ns, nonprime_avg_ns])

    rows.append({
        "function": f.__name__,
        "accuracy_%": (prime_acc + nonprime_acc) / 2.0,
        "prime_avg_ms": prime_avg_ns / 1e6,
        "nonprime_avg_ms": nonprime_avg_ns / 1e6,
        "both_avg_ms": both_avg_ns / 1e6,
    })

df = pd.DataFrame(rows)
df = df.sort_values(by="prime_avg_ms", ascending=True).reset_index(drop=True)
df

c:\Users\Ayrton\Documents\Gitprojects\DeepPrime\.env\Lib\site-packages\pyprimes.py:781: UserWarning: number is only probably prime not certainly prime
  warnings.warn("number is only probably prime not certainly prime")


,function,accuracy_%,prime_avg_ms,nonprime_avg_ms,both_avg_ms
0,is_prime_gmpy2,100.0,0.00811,0.00087,0.004490
1,is_prime_sympy,100.0,0.02181,0.00173,0.011770
2,is_prime_primefac,100.0,0.10665,0.00310,0.054875
3,is_prime_pycryptodome,100.0,1.89681,0.00174,0.949275
4,is_prime_pyprimes,100.0,2.00999,0.04431,1.027150
